In [2]:
import numpy as np
import pandas as pd
import math as m
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
# Testing..attention please..nuisance, you sent, who sent for me?

In [3]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states( coeff = [1/(2**.5),1/(2**.5),1/(2**.5),1/(2**.5)] , Abstract = False):    # coeff list like [a0,a1,b0,b1]
    if Abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [30]:
psi0 = Creating_states(Abstract=False)[0]
psi1 = Creating_states(Abstract=False)[1]

psi

[0.7071067811865475, 0.7071067811865475]

In [5]:
psi0sq = []
psi1sq = []
[[psi0sq.append(i*j) for i in psi0] for j in psi0]
[[psi1sq.append(i*j) for i in psi1] for j in psi1]   # creating list of coeff for squared states (intermeditate step)

psi0psi0 = [psi0sq[0], (psi0sq[1]**2+psi0sq[2]**2)**(.5), psi0sq[3]]        # the second element is the normalized coefficient due to the addition of states (01+10). See OneNote writeup for details
psi1psi1 = [psi1sq[0], (psi1sq[1]**(2)+psi1sq[2]**(2))**(.5), psi1sq[3]]            # symmetric states in basis (|00>, normalized version of (|01>+|10>),|11>)

# psi0psi0
# sum([i**2 for i in psi0psi0])   # adding to 1 now.

In [6]:
#creating the density matrix rho
# rho is 1/2*(|psi0psi0Xpsi0psi0|+|psi1psi1Xpsi1psi1|)
# findiing the conjugate vectors 
vec_psi0psi0 = sym.Matrix(psi0psi0)     # convertying to sympy vectors for easy matrix multiplication
vec_psi1psi1 = sym.Matrix(psi1psi1)

rho = (1/2)*(vec_psi0psi0*vec_psi0psi0.T  + vec_psi1psi1*vec_psi1psi1.T)       # density matrix with priors 1/2 each.
rho

Matrix([
[             0.25, 0.353553390593274,              0.25],
[0.353553390593274,               0.5, 0.353553390593274],
[             0.25, 0.353553390593274,              0.25]])

In [20]:
# creating the SIC POVM matrices
w = m.e**((2/3)*m.pi*(1j))
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]
# sum([np.trace(POVM_elts[i]) for i in range(9)])`# trace sum checks out`

In [47]:
# prob_vec = [(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] #[(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
# prob_vec
# sum(prob_vec)        # prob not adding to 1?
print(prob_vec)
(POVM_elts[4])

[0.00714886980224206, 0, 0.00714886980224206, 0.0660744349011210, 0.125000000000000, 0.183925565098879, 0.183925565098879, 0.125000000000000, 0.183925565098879]


array([[ 0.16666667+0.00000000e+00j,  0.        +0.00000000e+00j,
         0.08333333+1.44337567e-01j],
       [ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
        -0.        +0.00000000e+00j],
       [ 0.08333333-1.44337567e-01j, -0.        +0.00000000e+00j,
         0.16666667-7.80617690e-19j]])

In [65]:
np.trace(POVM_elts[4])

(0.08333333333333344+0.14433756729740643j)

In [18]:
# verifying that the trace still holds to be the same, equalt to .111 for this weird POVM element.
c = .00308641975308642*2.71828182845905**(4.18879020478639*(sym.I))

p = cm.polar(c)
cm.rect(p[0], p[1])
 
r = .00617-.0015
i = .0026

ab = (r**2+i**2)**(.5)
2*ab # verified

0.010689976613632043

In [19]:
N = 90          # number of measurements

POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']       # symbols to indicate collapsed direction

#prob distribution is simply the corresponding elements of the prob_vec
collapse_dir = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)

nj_list = [collapse_dir.count(f'd{i}') for i in range(9)]
print(prob_vec)
print(collapse_dir)

nj_list


TypeError: cannot determine truth value of Relational